In [1]:
!pip install -q py-feat
# !pip install -q facial-emotion-recognition
# !pip install -q fer
# !pip install -q deepface
# !pip install -q mtcnn

     |████████████████████████████████| 6.0MB 9.4MB/s 
     |████████████████████████████████| 3.3MB 33.9MB/s 
     |████████████████████████████████| 57.4MB 92kB/s 
     |████████████████████████████████| 4.9MB 41.1MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from feat import Detector
import glob
import math
import dlib
import joblib
import cv2
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from collections import Counter

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [ ]:
################# py-feat
face_model = "retinaface"
landmark_model = "MobileNet"
au_model = "svm"
emotion_model = "resmasknet" #resmasknet,fer, svm, rf
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

In [12]:
ls 'drive/MyDrive/file2/vdo/Isaree/02_Bell.mp4'

drive/MyDrive/file2/vdo/Isaree/02_Bell.mp4


In [6]:
ls 'drive/MyDrive/file2/done'

face2_01_KARNMAY_P1.csv       face6_01_mintchyy_ep2_P1.csv
face2_01_KARNMAY.xlsx         face6_01_mintchyy_ep2.xlsx
face2_02_Kemisara_P1.csv      face6_02_Babyjingko_P1.csv
face2_02_Kemisara.xlsx        face6_02_Babyjingko.xlsx
face5_01_jaysbabyfood_P1.csv  face7_01_mintchyy_QA_100_P1.csv
face5_01_jaysbabyfood.xlsx    face7_01_mintchyy_QA_100.xlsx
face5_02_gssspotted_P1.csv    face_7_2_01_thematter_P1.csv
face5_02_gssspotted.xlsx      face_7_2_01_thematter.xlsx
face5_03_clubfriday_P1.csv    old/
face5_03_clubfriday.xlsx


In [ ]:
from shutil import copyfile, copy
filepath = 'drive/MyDrive/file2/done/face2_02_Kemisara_P1.csv'
vdopath = 'drive/MyDrive/file2/vdo/Isaree/02_Bell.mp4'
df = pd.read_csv(filepath)
dictT = {'Neutral':'neutral', 'มีความสุข':'happy', 'ผ่อนคลาย':'neutral', 'เครียด':'angry', 'เศร้า':'sad',
         'มีความสุช':'happy', 'โกรธ':'angry', 'มีความสุุข':'happy', 'มีความสูข':'happy', 'ผ่่อนคลาย':'neutral',
         'ตื่นเต้น':'happy', 'ประหลาดใจ':'surprise', 'จริงจัง':'angry', 'neutral ': 'neutral', 'ผ่อ่นคลาย':'neutral',
         'มีตวามสุข':'happy', 'โกรธ ':'angry'}

df['done'] = df['done'].replace(dictT)
print([pd.unique(df['done'])])

for i in tqdm(range(df.shape[0])):
  for j in range(df['start_frame'][i], df['end_frame'][i],5):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    faceimg = detector.detect_faces(image)
    try:
      pos = faceimg[0][:4] # [FaceRectX, FaceRectY, FaceRectWidth, FaceRectHeight]
      cropimg =image[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] #[x, y, width, height]  
      dim = (224, 224)
      resized = cv2.resize(cropimg, dim)
      # cv2_imshow(resized)
      filename = 'lab.png'
      cv2.imwrite(filename, resized)
      emotion = df['done'][i]
      dst = 'lab'+str(j)+'.png'
      terminal_path = "drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/face_datalab_done/{0}/{1}".format(emotion, dst)
      copyfile('/content/lab.png', terminal_path)
    except:
      pass

[array(['happy', 'neutral', 'angry'], dtype=object)]
